In [ ]:
scaler = StandardScaler()
X_train_escalado = scaler.fit_transform(X_train)
X_test_escalado = scaler.transform(X_test)

# 1.7 GRID SEARCH PARA LOS PARAMETROS SOLICITADOS DE LAS SVM

## 1.6 Costo con Kernel lineal

In [ ]:
svm_params = SVC(random_state=42)

# Definir los hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear'],    
}


grid_search = GridSearchCV(estimator=svm_params, param_grid=param_grid, scoring = ['accuracy', 'precision_macro', 'recall_macro'], refit='accuracy')


grid_search.fit(X_train_escalado, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print('Mejores hiperparámetros:', best_params)

## 1.7 Costo y gamma con kernel gaussiano

In [ ]:
# Definición de un modelo clasificador de Random Forest
svm_params = SVC(random_state=42)

# Definir los hiperparámetros a ajustar y sus posibles valores
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf'],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100],

}

# Configurar la búsqueda grid con validación cruzada (cross-validation)
grid_search = GridSearchCV(estimator=svm_params, param_grid=param_grid, scoring = ['accuracy', 'precision_macro', 'recall_macro'], refit='accuracy')

# Realizar la búsqueda grid en el conjunto de entrenamiento
grid_search.fit(X_train_escalado, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print('Mejores hiperparámetros:', best_params)

# 2.Realizar la predicción del atributo Color utilizando máquinas de vectores con kernel lineal analizando el parámetro costo

## 2.1 SVM Con Kernel lineal y analizando el parametro costo SIN CROSSVALIDATION


In [ ]:
svm_linear_nocross = SVC(kernel='linear', C=0.1, random_state=42)

In [ ]:
svm_linear_nocross.fit(X_train_escalado, y_train)

In [ ]:
# Predicciones
y_pred_linear = svm_linear_nocross.predict(X_test_escalado)

# Evaluación
accuracy = accuracy_score(y_test, y_pred_linear)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
dataframe['Color'].value_counts()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


cm = (confusion_matrix(y_test, y_pred_linear))
a = ["Blue-Green", "Bluish-Green", "Green"]

sns.heatmap(cm, xticklabels = a, yticklabels = a,annot=True, fmt='g')

print(classification_report(y_test, y_pred_linear))

## 2.2 Kernel lineal con cross validation

In [ ]:
svm = SVC(kernel='linear', C=0.1)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
fold = 1
for i, (train_index, val_index) in enumerate(kf.split(X_train_escalado, y_train)):
    
    #evitar el error out of index
    index_labels = y_train.index
    trainsc_df = pd.DataFrame(X_train_escalado, index=index_labels)


    X_train_fold, X_val_fold = trainsc_df.iloc[train_index], trainsc_df.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training fold
    svm.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_fold = svm.predict(X_val_fold)

    # Compute confusion matrix and classification report
    cm = confusion_matrix(y_val_fold, y_pred_fold)
    # Matriz de confusion x cada fold
    a = ["Blue-Green", "Bluish-Green", "Green"]
    sns.heatmap(cm, xticklabels=a, yticklabels=a, annot=True, fmt='g')
    plt.title(f"Confusion Matrix para fold numero {fold}")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Print classification report
    print(f"Classification Report para fold numero {fold}:\n", classification_report(y_val_fold, y_pred_fold))

    fold += 1

- El modelo generaliza igual sobre el conjunto de test
- ambos modelos (con y sin Cross validation) tienen recall de 1 en la clase green
- Todos los datos de green se predicen correctamente
- Se predicen como green algunos otros valores, pero como la cantidad de green es tan alta la accuracy es de 85% lo cual es alta


# 3. Realizar la predicción del atributo Color utilizando máquinas de vectores con kernelgaussiano analizando los parámetros costo y gama


## 3.1 SVM con kernel gaussiano sin cross validation

In [ ]:
svm_gauss_nocross = SVC(kernel='rbf', C=0.1, random_state=42, gamma=0.001)

In [ ]:
svm_gauss_nocross.fit(X_train_escalado, y_train)

In [ ]:
# Predicciones
y_pred_gauss = svm_linear_nocross.predict(X_test_escalado)

# Evaluación
accuracy = accuracy_score(y_test, y_pred_gauss)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
dataframe['Color'].value_counts()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


cm = (confusion_matrix(y_test, y_pred_gauss))
a = ["Blue-Green", "Bluish-Green", "Green"]

sns.heatmap(cm, xticklabels = a, yticklabels = a,annot=True, fmt='g')

print(classification_report(y_test, y_pred_gauss))

## 3.2 SVM con Kernel gaussiano y cross validation

In [ ]:
svm_gaussiano = SVC(kernel='rbf', C=0.1, random_state=42, gamma=0.001)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
fold = 1
for i, (train_index, val_index) in enumerate(kf.split(X_train_escalado, y_train)):
    
    #evitar el error out of index
    index_labels = y_train.index
    trainsc_df = pd.DataFrame(X_train_escalado, index=index_labels)


    X_train_fold, X_val_fold = trainsc_df.iloc[train_index], trainsc_df.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training fold
    svm_gaussiano.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_fold = svm_gaussiano.predict(X_val_fold)

    # Compute confusion matrix and classification report
    cm = confusion_matrix(y_val_fold, y_pred_fold)
    # Matriz de confusion x cada fold
    a = ["Blue-Green", "Bluish-Green", "Green"]
    sns.heatmap(cm, xticklabels=a, yticklabels=a, annot=True, fmt='g')
    plt.title(f"Confusion Matrix para fold numero {fold}")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Print classification report
    print(f"Classification Report para fold numero {fold}:\n", classification_report(y_val_fold, y_pred_fold))



    fold += 1

# 4. Realizar la predicción del atributo Color utilizando Random Forest analizando los parámetros cantidad de estimadores y la máxima profundidad de los árboles

## 4.1 random forest sin cross validation

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest_nocross = RandomForestClassifier(random_state=42)

In [ ]:
forest_nocross.fit(X_train_escalado, y_train)

In [ ]:
# Predicciones
y_pred_forest = forest_nocross.predict(X_test_escalado)

# Evaluación
accuracy = accuracy_score(y_test, y_pred_forest)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


cm = (confusion_matrix(y_test, y_pred_linear))
a = ["Blue-Green", "Bluish-Green", "Green"]

sns.heatmap(cm, xticklabels = a, yticklabels = a,annot=True, fmt='g')

print(classification_report(y_test, y_pred_linear))

## 4.2 Random forest con cross validation e hiperparametros ajustados mediante grid search

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)

# Definir los hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [30, 50],

}


grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, scoring = ['accuracy', 'precision_macro', 'recall_macro'], refit='accuracy')


grid_search.fit(X_train_escalado, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print('Mejores hiperparámetros:', best_params)

Mejores hiperparámetros: {'class_weight': 'balanced', 'max_depth': 30, 'max_features': 'sqrt', 'max_leaf_nodes': 300, 'max_samples': 50, 'min_samples_leaf': 10, 'n_estimators': 50, 'random_state': 42}

In [ ]:
rand_forest = RandomForestClassifier(class_weight='balanced',max_depth= 30, max_features='sqrt', max_leaf_nodes= 300, max_samples= 50, min_samples_leaf= 10, n_estimators= 50, random_state=42)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
fold = 1
for i, (train_index, val_index) in enumerate(kf.split(X_train_escalado, y_train)):
    
    #evitar el error out of index
    index_labels = y_train.index
    trainsc_df = pd.DataFrame(X_train_escalado, index=index_labels)


    X_train_fold, X_val_fold = trainsc_df.iloc[train_index], trainsc_df.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training fold
    rand_forest.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_fold = rand_forest.predict(X_val_fold)

    # Compute confusion matrix and classification report
    cm = confusion_matrix(y_val_fold, y_pred_fold)
    # Matriz de confusion x cada fold
    a = ["Blue-Green", "Bluish-Green", "Green"]
    sns.heatmap(cm, xticklabels=a, yticklabels=a, annot=True, fmt='g')
    plt.title(f"Confusion Matrix para fold numero {fold}")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Print classification report
    print(f"Classification Report para fold numero {fold}:\n", classification_report(y_val_fold, y_pred_fold))



    fold += 1